## Seoul_Accident_Hazard

In [5]:
import json
geo_path = 'Triple_Seoul.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str

# 데이터 정리 
import numpy as np
import pandas as pd
Accident_Hazard = pd.read_csv('Seoul_Accident_Hazard.csv', encoding='euc-kr', thousands=',')

Accident_Hazard = Accident_Hazard[['시도시군구명', '발생건수', '경도', '위도']]
Accident_Hazard = Accident_Hazard.rename(columns={'시도시군구명':'구별'})

Accident_Hazard = Accident_Hazard[0:75]
Accident = Accident_Hazard[['구별', '발생건수', '경도', '위도']]

Accident.set_index('구별', inplace=True)

import folium
import pandas as pd

map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='openstreetmap')
# tiles= 'openstreetmap', 'cartodbpositron', 'stamen Toner'
soo = folium.Choropleth(geo_data=geo_str,
               data = Accident['발생건수'],
               columns = [Accident.index, Accident['발생건수']],
               threshold_scale= 8,
               fill_color='PuRd',
               key_on='feature.properties.name',
               highlight=True,
               fill_opacity=0.5, line_opacity=1).add_to(map)

folium.GeoJsonTooltip(fields = ['name']).add_to(soo.geojson)
map

## + 기후에 따른 교통사고 합계

In [6]:
import json
geo_path2 = 'Seoul2.json'
geo_str2 = json.load(open(geo_path2, encoding='utf-8'))

# 데이터 정리 
import numpy as np
import pandas as pd
weather = pd.read_csv('기상상태별_교통사고_18년.csv', encoding='euc-kr', thousands=',')

weather.rename(columns={weather.columns[1] : '구별'}, inplace=True)

occur = weather[3:78:3]
clear_occur = occur[['구별', '구분', '계']]
clear_occur = clear_occur.reset_index()
clear_occur = clear_occur[['구별', '구분', '계']]

clear_occur.set_index('구별', inplace=True)

from sklearn import preprocessing
col = ['계']
x = clear_occur[col].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.astype(float))
clear_occur = pd.DataFrame(x_scaled, columns = col, index = clear_occur.index)

import folium
import pandas as pd

map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                tiles='cartodbpositron')
# tiles='cartodbpositron'
soo = folium.Choropleth(geo_data=geo_str,
               data = Accident['발생건수'],
               columns = [Accident.index, Accident['발생건수']],
               fill_color='BuPu',
               key_on='feature.properties.name',
               highlight=True,
               fill_opacity=0.5, line_opacity=1).add_to(map)

# 기후에 따른 교통사고 합계
map.choropleth(geo_data=geo_str2,
               data = clear_occur['계'],
               columns = [clear_occur.index, clear_occur['계']],
               fill_color='YlGn',
               key_on='feature.properties.name',
               highlight=True,
               fill_opacity=0.3, line_opacity=3)

import json
with open('Seoul2.json',mode='rt',encoding='utf-8') as f:
    geo = json.loads(f.read())
    f.close()

folium.GeoJsonTooltip(fields = ['name']).add_to(soo.geojson)

map

C:\ProgramData\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


## 사고다발지역과 도로 folium

In [7]:
# import os
# import folium
# import pandas as pd
# import numpy as np
# import networkx as nx

# nodes = pd.read_csv('Seoul_nodes_190709.csv')
# links = pd.read_csv('Seoul_links_190709.csv')

# nodes = nodes[['Id','NODE_NAME','latitude','longitude']]

# links = links[['Source','Target']]
# links.head()

# source_in = links['Source'].apply(lambda x : x in list(nodes['Id'])) # check Sources are in incheon_id
# target_in = links['Target'].apply(lambda x : x in list(nodes['Id'])) # check Targets are in incheon_id
# # source_in and target_in are boolean type pandas.Series which contains True or False
# Seoul_links = links[source_in & target_in] # contain if both target and source are contained in incheon_id

# # Positioning the Standard Point for our Folium Map
# std_point = tuple(nodes.head(1)[['latitude','longitude']].iloc[0])
# std_point

# map_osm = folium.Map(location=std_point, zoom_start=10) 
# # location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

# map_osm.choropleth(geo_data=geo_str,
#                    data = Accident['발생건수'],
#                    columns = [Accident.index, Accident['발생건수']],
#                    fill_color='PuRd',
#                    key_on='feature.properties.name',
#                    highlight=True,
#                    fill_opacity=0.5, line_opacity=1)

# kw = {'opacity': 0.5, 'weight': 2}
# for ix, row in Seoul_links.iterrows():
#     start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
#     end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
#     folium.PolyLine(
#         locations=[start, end],
#         color='blue',
#         line_cap='round',
#         **kw,
#     ).add_to(map_osm)
# # it takes some time.....
# map_osm.save('Accident_hazard_road.html')

In [8]:
import os
import folium
import pandas as pd
import numpy as np
import networkx as nx

nodes = pd.read_csv('Seoul_nodes_190709.csv')
links = pd.read_csv('Seoul_links_190709.csv')

nodes = nodes[['Id','NODE_NAME','latitude','longitude']]

links = links[['Source','Target']]
links.head()

source_in = links['Source'].apply(lambda x : x in list(nodes['Id'])) # check Sources are in incheon_id
target_in = links['Target'].apply(lambda x : x in list(nodes['Id'])) # check Targets are in incheon_id
# source_in and target_in are boolean type pandas.Series which contains True or False
Seoul_links = links[source_in & target_in] # contain if both target and source are contained in incheon_id

# Positioning the Standard Point for our Folium Map
std_point = tuple(nodes.head(1)[['latitude','longitude']].iloc[0])
std_point

map_osm = folium.Map(location=std_point, zoom_start=10) 
# location : 기준이 되는 점, zoom_start : 지도 상의 zoom level 을 나타냅니다.

kw = {'opacity': 0.5, 'weight': 2}
for ix, row in Seoul_links.iterrows():
    start = tuple(nodes[nodes['Id']==row['Source']][['latitude','longitude']].iloc[0])
    end = tuple(nodes[nodes['Id']==row['Target']][['latitude','longitude']].iloc[0])
    folium.PolyLine(
        locations=[start, end],
        color='blue',
        line_cap='round',
        **kw,
    ).add_to(map_osm)
    
soo = folium.Choropleth(geo_data=geo_str,
                   data = Accident['발생건수'],
                   columns = [Accident.index, Accident['발생건수']],
                   fill_color='PuRd',
                   key_on='feature.properties.name',
                   highlight=True,
                   fill_opacity=0.5, line_opacity=1).add_to(map_osm)

folium.GeoJsonTooltip(fields = ['name']).add_to(soo.geojson)
    
map_osm.save('Accident_Hazard_road.html')